# Protein Folding | EPOCH Lab 2022

In [1]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

import nglview as nv

In [2]:
try:
    platform = Platform.getPlatformByName("CUDA")
except Exception:
    platform = Platform.getPlatformByName("OpenCL")

In [3]:
pdb = PDBFile('pdb/input.pdb')
forcefield = ForceField('amber99sb.xml', 'tip3p.xml')

In [4]:
modeller = Modeller(pdb.topology, pdb.positions)
# modeller.addHydrogens(forcefield)
modeller.topology

<Topology; 1 chains, 2798 residues, 8867 atoms, 6111 bonds>

In [5]:
unmatched_residues = forcefield.getUnmatchedResidues(modeller.topology)
unmatched_residues

[]

In [6]:
system = forcefield.createSystem(modeller.topology,
                                 nonbondedMethod=PME,
                                 nonbondedCutoff=1*nanometer,
                                 constraints=HBonds)

In [7]:
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)

In [8]:
simulation = Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('/tmp/output.pdb', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True, potentialEnergy=True, temperature=True))

In [9]:
simulation.step(10000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000,-124389.91141541512,251.26102950530648
2000,-117876.97391541512,283.8223793219209
3000,-115399.75516541512,295.6022105087631
4000,-115659.16141541512,298.6641407401495
5000,-115608.16141541512,300.54441207260265
6000,-115763.56766541512,297.28625628512356
7000,-115442.31766541512,294.95534060475615
8000,-115630.94266541512,295.94394783976696
9000,-115604.50516541512,298.2841683518243
10000,-115149.72391541512,303.8256980679343


In [10]:
view = nv.show_file('/tmp/output.pdb')
view

NGLWidget()